In [1]:
import spark.implicits._
val df = spark.read.option("sep",";").option("header","true").option("inferSchema","true").option("ignoreTrailingWhiteSpace","true")
    .csv("/home/jovyan/LearningSpark2/resources/Rango_Edades_Seccion_202203.csv").na.fill(0)
    .withColumn("DESC_DISTRITO",trim($"DESC_DISTRITO")).withColumn("DESC_BARRIO",trim($"DESC_BARRIO"))

Intitializing Scala interpreter ...

Spark Web UI available at http://725a4c8e5c14:4040
SparkContext available as 'sc' (version = 3.2.1, master = local[*], app id = local-1647506543044)
SparkSession available as 'spark'


import spark.implicits._
df: org.apache.spark.sql.DataFrame = [COD_DISTRITO: int, DESC_DISTRITO: string ... 10 more fields]


In [2]:
df.show(5)
df.printSchema()

+------------+-------------+---------------+-----------+----------+----------------+-----------+------------+----------------+----------------+------------------+------------------+
|COD_DISTRITO|DESC_DISTRITO|COD_DIST_BARRIO|DESC_BARRIO|COD_BARRIO|COD_DIST_SECCION|COD_SECCION|COD_EDAD_INT|EspanolesHombres|EspanolesMujeres|ExtranjerosHombres|ExtranjerosMujeres|
+------------+-------------+---------------+-----------+----------+----------------+-----------+------------+----------------+----------------+------------------+------------------+
|           1|       CENTRO|            101|    PALACIO|         1|            1001|          1|           0|               3|               1|                 1|                 0|
|           1|       CENTRO|            101|    PALACIO|         1|            1001|          1|           1|               5|               2|                 1|                 0|
|           1|       CENTRO|            101|    PALACIO|         1|            1001|      

In [3]:
//Select distinct neighborhood
val barriosDf = df.select($"DESC_BARRIO").distinct
barriosDf.show()

+-----------------+
|      DESC_BARRIO|
+-----------------+
|    LOS JERONIMOS|
|       NI�O JESUS|
|         VALVERDE|
|           CORTES|
|PALOMERAS SURESTE|
|        TRAFALGAR|
|           HELLIN|
|     VISTA ALEGRE|
|        FONTARRON|
|           VENTAS|
| FUENTE DEL BERRO|
|           LUCERO|
|         PACIFICO|
|   CUATRO CAMINOS|
|CASCO H.VICALVARO|
|         EL PILAR|
|        ARGUELLES|
|    VALDEACEDERAS|
|         CANILLAS|
|     VALLEHERMOSO|
+-----------------+
only showing top 20 rows



barriosDf: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [DESC_BARRIO: string]


In [4]:
//Create table view
df.createOrReplaceTempView("padron")

In [5]:
//Select distinct neighborhood from temp view
spark.sql("SELECT DISTINCT(DESC_BARRIO) FROM padron").show()

+-----------------+
|      DESC_BARRIO|
+-----------------+
|    LOS JERONIMOS|
|       NI�O JESUS|
|         VALVERDE|
|           CORTES|
|PALOMERAS SURESTE|
|        TRAFALGAR|
|           HELLIN|
|     VISTA ALEGRE|
|        FONTARRON|
|           VENTAS|
| FUENTE DEL BERRO|
|           LUCERO|
|         PACIFICO|
|   CUATRO CAMINOS|
|CASCO H.VICALVARO|
|         EL PILAR|
|        ARGUELLES|
|    VALDEACEDERAS|
|         CANILLAS|
|     VALLEHERMOSO|
+-----------------+
only showing top 20 rows



In [6]:
//Create a column with length of DESC_BARRIO
val lengthBarrio = barriosDf.withColumn("LONGITUD",length($"DESC_BARRIO"))
lengthBarrio.show()

+-----------------+--------+
|      DESC_BARRIO|LONGITUD|
+-----------------+--------+
|    LOS JERONIMOS|      13|
|       NI�O JESUS|      10|
|         VALVERDE|       8|
|           CORTES|       6|
|PALOMERAS SURESTE|      17|
|        TRAFALGAR|       9|
|           HELLIN|       6|
|     VISTA ALEGRE|      12|
|        FONTARRON|       9|
|           VENTAS|       6|
| FUENTE DEL BERRO|      16|
|           LUCERO|       6|
|         PACIFICO|       8|
|   CUATRO CAMINOS|      14|
|CASCO H.VICALVARO|      17|
|         EL PILAR|       8|
|        ARGUELLES|       9|
|    VALDEACEDERAS|      13|
|         CANILLAS|       8|
|     VALLEHERMOSO|      12|
+-----------------+--------+
only showing top 20 rows



lengthBarrio: org.apache.spark.sql.DataFrame = [DESC_BARRIO: string, LONGITUD: int]


In [7]:
//Create a column with 5 for each record and drop the column
val withColumn5Df = df.withColumn("5",lit("5"))
withColumn5Df.show(5)
withColumn5Df.drop($"5").show(5)

+------------+-------------+---------------+-----------+----------+----------------+-----------+------------+----------------+----------------+------------------+------------------+---+
|COD_DISTRITO|DESC_DISTRITO|COD_DIST_BARRIO|DESC_BARRIO|COD_BARRIO|COD_DIST_SECCION|COD_SECCION|COD_EDAD_INT|EspanolesHombres|EspanolesMujeres|ExtranjerosHombres|ExtranjerosMujeres|  5|
+------------+-------------+---------------+-----------+----------+----------------+-----------+------------+----------------+----------------+------------------+------------------+---+
|           1|       CENTRO|            101|    PALACIO|         1|            1001|          1|           0|               3|               1|                 1|                 0|  5|
|           1|       CENTRO|            101|    PALACIO|         1|            1001|          1|           1|               5|               2|                 1|                 0|  5|
|           1|       CENTRO|            101|    PALACIO|         1|   

withColumn5Df: org.apache.spark.sql.DataFrame = [COD_DISTRITO: int, DESC_DISTRITO: string ... 11 more fields]


In [8]:
//Partition table by DESC_DISTRITO, DESC_BARRIO
df.write.mode("overwrite").option("sep",";").option("header","true").partitionBy("DESC_DISTRITO","DESC_BARRIO").csv("/home/jovyan/LearningSpark2/output/padron/")

In [9]:
/*
Load df in cache
group by ditrict and neighborhood ,show total of SpanishMen,SpanishWomen,ForeingMen,ForeignWomen ,order by ForeignWomen,ForeingMen
Unload df of cache
*/
df.persist()
df.select($"DESC_DISTRITO",$"DESC_BARRIO",$"EspanolesHombres",$"EspanolesMujeres",$"ExtranjerosHombres",$"ExtranjerosMujeres")
    .groupBy($"DESC_DISTRITO",$"DESC_BARRIO")
    .agg(sum($"EspanolesHombres").as("SumEspanolesHombres")
         ,sum($"EspanolesMujeres").as("SumEspanolesMujeres")
         ,sum($"ExtranjerosHombres").as("SumExtranjerosHombres")
         ,sum($"ExtranjerosMujeres").as("SumExtranjerosMujeres"))
    .orderBy(desc("SumExtranjerosMujeres"),desc("SumExtranjerosHombres")).show(5)
df.unpersist()



+------------------+------------+-------------------+-------------------+---------------------+---------------------+
|     DESC_DISTRITO| DESC_BARRIO|SumEspanolesHombres|SumEspanolesMujeres|SumExtranjerosHombres|SumExtranjerosMujeres|
+------------------+------------+-------------------+-------------------+---------------------+---------------------+
|PUENTE DE VALLECAS|   SAN DIEGO|              13889|              15566|                 6935|                 7096|
|     CIUDAD LINEAL|PUEBLO NUEVO|              23112|              27183|                 5626|                 6415|
|            LATINA|      ALUCHE|              24824|              29557|                 5451|                 6386|
|       CARABANCHEL|VISTA ALEGRE|              15791|              19255|                 5553|                 6175|
|            CENTRO| EMBAJADORES|              16389|              16338|                 7983|                 5895|
+------------------+------------+-------------------+---

res7: df.type = [COD_DISTRITO: int, DESC_DISTRITO: string ... 10 more fields]


In [10]:
/*
Create a new DataFrame from df with DESC_DISTRITO,DESC_BARRIO,SumEspanolesHombres
Join with df
Do the same with window functions
*/
val newDf = df.select($"DESC_DISTRITO",$"DESC_BARRIO",$"EspanolesHombres")
        .groupBy($"DESC_DISTRITO",$"DESC_BARRIO")
        .agg(sum($"EspanolesHombres").as("SumEspanolesHombres"))
        
df.join(newDf,df("DESC_DISTRITO")<=>newDf("DESC_DISTRITO") && df("DESC_BARRIO")<=>newDf("DESC_BARRIO"))
    .drop(newDf("DESC_DISTRITO"))
    .drop(newDf("DESC_BARRIO"))
    .dropDuplicates()
    .orderBy($"COD_DISTRITO",$"COD_DIST_BARRIO")
    .show(4)
import org.apache.spark.sql.expressions.{Window, WindowSpec}
val window = Window.partitionBy($"DESC_DISTRITO",$"DESC_BARRIO")
df.withColumn("SumEspanolesHombres",sum($"EspanolesHombres").over(window))
    .orderBy($"COD_DISTRITO",$"COD_DIST_BARRIO")
    .show(4)

+------------+---------------+----------+----------------+-----------+------------+----------------+----------------+------------------+------------------+-------------+-----------+-------------------+
|COD_DISTRITO|COD_DIST_BARRIO|COD_BARRIO|COD_DIST_SECCION|COD_SECCION|COD_EDAD_INT|EspanolesHombres|EspanolesMujeres|ExtranjerosHombres|ExtranjerosMujeres|DESC_DISTRITO|DESC_BARRIO|SumEspanolesHombres|
+------------+---------------+----------+----------------+-----------+------------+----------------+----------------+------------------+------------------+-------------+-----------+-------------------+
|           1|            101|         1|            1004|          4|          37|               9|               5|                 2|                 7|       CENTRO|    PALACIO|               9197|
|           1|            101|         1|            1008|          8|          76|               2|               3|                 0|                 0|       CENTRO|    PALACIO|           

newDf: org.apache.spark.sql.DataFrame = [DESC_DISTRITO: string, DESC_BARRIO: string ... 1 more field]
import org.apache.spark.sql.expressions.{Window, WindowSpec}
window: org.apache.spark.sql.expressions.WindowSpec = org.apache.spark.sql.expressions.WindowSpec@47e2bc65


In [13]:
/*
Show sum of the EspanolesMujeres group by COD_EDAD_INT in BARAJAS,CENTRO,RETIRO
*/
val pivotDf = df.filter($"DESC_DISTRITO".equalTo("CENTRO") || $"DESC_DISTRITO".equalTo("BARAJAS") || $"DESC_DISTRITO".equalTo("RETIRO"))
    .groupBy($"COD_EDAD_INT")
    .pivot($"DESC_DISTRITO").sum("EspanolesMujeres")
    .orderBy($"COD_EDAD_INT")

pivotDf.show()

/*val percentOfTotalWomen = pivotDf.withColumn("potwBARAJAS",$"BARAJAS"/sum($"BARAJAS").over(window))
    .withColumn("potwCENTRO",$"CENTRO"/sum($"CENTRO"))
    .withColumn("potwRETIRO",$"RETIRO"/sum($"RETIRO"))*/
    


+------------+-------+------+------+
|COD_EDAD_INT|BARAJAS|CENTRO|RETIRO|
+------------+-------+------+------+
|           0|    148|   245|   298|
|           1|    157|   247|   325|
|           2|    166|   209|   366|
|           3|    181|   217|   365|
|           4|    206|   237|   420|
|           5|    211|   221|   408|
|           6|    241|   244|   406|
|           7|    236|   231|   462|
|           8|    232|   231|   423|
|           9|    240|   237|   422|
|          10|    261|   255|   405|
|          11|    242|   238|   427|
|          12|    256|   245|   424|
|          13|    277|   274|   437|
|          14|    244|   255|   411|
|          15|    260|   244|   417|
|          16|    276|   266|   426|
|          17|    261|   255|   415|
|          18|    231|   294|   456|
|          19|    241|   284|   415|
+------------+-------+------+------+
only showing top 20 rows



pivotDf: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [COD_EDAD_INT: int, BARAJAS: bigint ... 2 more fields]


In [ ]:
df.repartition($"")